In [ ]:
!pip freeze > requirements.txt

In [1]:
import os  
from GreenHouseEnv.envTHumidpHEC import GreenHousePCSEEnvVer2
from pcse_gym.envs.common_env import  PCSEEnv
import numpy as np

from tensorflow.summary import create_file_writer, scalar
from stable_baselines3 import A2C, SAC, PPO, TD3
from stable_baselines3.common.callbacks import BaseCallback
from datetime import datetime 
import matplotlib.pyplot as plt
import warnings

import tensorflow as tf
from tensorboard import program

In [ ]:
%load_ext tensorboard

In [2]:
warnings.filterwarnings("ignore")
class CustomCallback(BaseCallback):
    def __init__(self, verbose=0, writer=None):
        super(CustomCallback, self).__init__(verbose)
        self.rewards = []
        self.writer = writer  # Передайте SummaryWriter

    def _on_step(self) -> bool:
        if self.model.ep_info_buffer:
            reward = self.model.ep_info_buffer[0].get('r', 0)  # Получаем значение 'r' из первого элемента или 0, если 'r' отсутствует
        else:
            reward = 0  # Обработка случая, когда очередь пуста

        self.rewards.append(reward)

        # Запишите вознаграждение в SummaryWriter
        with self.writer.as_default():
            scalar("Reward", reward, step=self.num_timesteps)

        # Здесь вы также можете записывать другие параметры среды

        return True

    def close(self) -> None:
        pass 
env = GreenHousePCSEEnvVer2(  timestep=1)

##  Model trainings

PPO (Proximal Policy Optimization):
```python
total_timesteps (int): #Общее количество шагов времени, в течение которых будет произведено обучение.
callback (BaseCallback, optional):# Обратный вызов (callback) для записи промежуточных результатов обучения.
log_interval (int):# Интервал для записи логов.
eval_env (gym.Env, optional): #Среда для оценки производительности модели.
eval_freq (int): #Частота оценки (в терминах количества шагов).
n_eval_episodes (int): #Количество эпизодов для оценки модели.
#Множество других аргументов, связанных с настройками обучения и гиперпараметрами.
```


SAC (Soft Actor-Critic):
```python
total_timesteps (int): # Общее количество шагов времени, в течение которых будет произведено обучение.
callback (BaseCallback, optional):#  Обратный вызов (callback) для записи промежуточных результатов обучения.
log_interval (int): #Интервал для записи логов.
eval_env (gym.Env, optional): #Среда для оценки производительности модели.
eval_freq (int): #Частота оценки (в терминах количества шагов).
n_eval_episodes (int): #Количество эпизодов для оценки модели.
# Множество других аргументов, связанных с настройками обучения и гиперпараметрами.
```

TD3 (Twin Delayed Deep Deterministic Policy Gradient):
```python
total_timesteps (int): # Общее количество шагов времени, в течение которых будет произведено обучение.
callback (BaseCallback, optional): #Обратный вызов (callback) для записи промежуточных результатов обучения.
log_interval (int):# Интервал для записи логов.
eval_env (gym.Env, optional):# Среда для оценки производительности модели.
eval_freq (int): #Частота оценки (в терминах количества шагов).
n_eval_episodes (int): #Количество эпизодов для оценки модели.
```

In [3]:
TENSORBOARD_LOGS_DIR=os.path.join(
    os.getcwd(),"tensorboard_logs"
    )
if not os.path.exists(TENSORBOARD_LOGS_DIR):
    os.makedirs(TENSORBOARD_LOGS_DIR)
ENV_DIR = os.path.join(TENSORBOARD_LOGS_DIR,env.__class__.name)
if not os.path.exists(ENV_DIR):
    os.makedirs(ENV_DIR)

### 4 models

In [ ]:
%load_ext tensorboard --logdir={MODEL_DIR}

In [4]:
total_timesteps = 5e4
models_variants = {
    'sac': {
    'batch_size': 64,
    'ent_coef': 'auto',
    'learning_rate': 0.0003,  # Снижаем learning rate
    'learning_starts': 1000,  # Увеличиваем learning_starts
    'buffer_size': int(1/4*total_timesteps),
},
    'a2c': {
    'n_steps': 5,
    'ent_coef': 0.01,
    'learning_rate': 0.0007,
    'vf_coef': 0.25,
    'max_grad_norm': 0.5,
},
    'td3': {
    "learning_rate": 1e-4,  # Увеличиваем learning rate
    "buffer_size": 1000000,  # Увеличиваем buffer_size
    "learning_starts": 1000,  # Увеличиваем learning_starts
    "batch_size": 128,  # Увеличиваем batch_size
    "tau": 0.005,  # Уменьшаем tau
    "gamma": 0.99,  # Увеличиваем gamma
    "train_freq": (1, "episode"),  # Увеличиваем train_freq
    "gradient_steps": 100,  # Увеличиваем gradient_steps
    "policy_delay": 2,  # Уменьшаем policy_delay
    "target_policy_noise": 0.2,  # Увеличиваем target_policy_noise
    "target_noise_clip": 0.5,  # Увеличиваем target_noise_clip
    "stats_window_size": 100,
}
}

In [5]:
date = datetime.now().strftime("%Y-%m-%d__%H-%M-%S")
logdirs = []
name_model = f'diff_models'

MODEL_DIR = os.path.join(ENV_DIR,name_model)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

for var in models_variants.keys():
    name_model_var= f'var_{var}'

    MODEL_DIR_VAR = os.path.join(MODEL_DIR,name_model_var)
    if not os.path.exists(MODEL_DIR_VAR):
        os.makedirs(MODEL_DIR_VAR)
    
    model_hyperparams = models_variants[var]
    match var:
        case 'sac':
            model = SAC("MultiInputPolicy", env, verbose=2,**model_hyperparams)
        case 'a2c':
            model = A2C("MultiInputPolicy", env, verbose=2,**model_hyperparams)
        case 'td3':
            model = TD3("MultiInputPolicy", env, verbose=2,**model_hyperparams)
            
    # Создайте директорию для записи данных TensorBoard
    log_dir = os.path.join(
        MODEL_DIR_VAR,
        date
        )  # Замените это на путь к вашей директории
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    # Создайте SummaryWriter
    writer = create_file_writer(log_dir)

    model.learn(
        total_timesteps=total_timesteps, # Общее количество шагов времени, в течение которых будет произведено обучение.
        callback = CustomCallback(writer=writer),#  Обратный вызов (callback) для записи промежуточных результатов обучения.
        log_interval=1e1, #Интервал для записи логов.
        progress_bar=True
    )
    model.save(path=os.path.join(MODEL_DIR_VAR,f'{var}'))
    logdirs.append(log_dir)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


KeyboardInterrupt: 

### SAC model

In [ ]:
sac_variants = {
    0: {'batch_size': 64, 'ent_coef': 'auto','learning_rate': 0.003,'learning_starts': 100,'buffer_size': 100000,},
    # 1: {'batch_size': 64, 'ent_coef': 'auto', 'learning_rate': 0.001, 'learning_starts': 100, 'buffer_size': 100000},
    # 2: {'batch_size': 64, 'ent_coef': 0.01, 'learning_rate': 0.003, 'learning_starts': 500, 'buffer_size': 50000},
    # 3: {'batch_size': 32, 'ent_coef': 0.005, 'learning_rate': 0.002, 'learning_starts': 200, 'buffer_size': 150000},
}

In [ ]:
date = datetime.now().strftime("%Y-%m-%d__%H-%M-%S")
logdirs = []
name_model = f'SAC'

MODEL_DIR = os.path.join(ENV_DIR,name_model)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

for var in sac_variants.keys():
    name_model_var= f'var_{var}'

    MODEL_DIR_VAR = os.path.join(MODEL_DIR,name_model_var)
    if not os.path.exists(MODEL_DIR_VAR):
        os.makedirs(MODEL_DIR_VAR)

    sac_hyperparams = sac_variants[var]

    model_sac = SAC("MultiInputPolicy", env, verbose=2,**sac_hyperparams)
    # Создайте директорию для записи данных TensorBoard
    log_dir = os.path.join(
        MODEL_DIR_VAR,
        date
        )  # Замените это на путь к вашей директории
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    # Создайте SummaryWriter
    writer = create_file_writer(log_dir)

    model_sac.learn(
        total_timesteps=1.5e6, # Общее количество шагов времени, в течение которых будет произведено обучение.
        callback = CustomCallback(writer=writer),#  Обратный вызов (callback) для записи промежуточных результатов обучения.
        log_interval=1e2, #Интервал для записи логов.
        progress_bar=True
    )
    model_sac.save(path=os.path.join(MODEL_DIR_VAR,f'SAC_{var}'))
    logdirs.append(log_dir)

In [ ]:
date = datetime.now().strftime("%Y-%m-%d__%H-%M-%S")
logdirs = []
name_model = f'SAC'

MODEL_DIR = os.path.join(ENV_DIR,name_model)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

for var in sac_variants.keys():
    name_model_var= f'var_{var}'

    MODEL_DIR_VAR = os.path.join(MODEL_DIR,name_model_var)
    if not os.path.exists(MODEL_DIR_VAR):
        os.makedirs(MODEL_DIR_VAR)

    sac_hyperparams = sac_variants[var]

    model_sac = SAC("MultiInputPolicy", env, verbose=2,**sac_hyperparams)
    # Создайте директорию для записи данных TensorBoard
    log_dir = os.path.join(
        MODEL_DIR_VAR,
        date
        )  # Замените это на путь к вашей директории
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    # Создайте SummaryWriter
    writer = create_file_writer(log_dir)

    model_sac.learn(
        total_timesteps=1.5e6, # Общее количество шагов времени, в течение которых будет произведено обучение.
        callback = CustomCallback(writer=writer),#  Обратный вызов (callback) для записи промежуточных результатов обучения.
        log_interval=1e2, #Интервал для записи логов.
        progress_bar=True
    )
    model_sac.save(path=os.path.join(MODEL_DIR_VAR,f'SAC_{var}'))
    logdirs.append(log_dir)

In [ ]:
%load_ext tensorboard --logdir={MODEL_DIR}

### A2C model

In [ ]:
a2c_variants = {
    0: {'n_steps': 5,'ent_coef': 0.01,'learning_rate': 0.0007,'vf_coef': 0.25,'max_grad_norm': 0.5,},
    1: {'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.001, 'vf_coef': 0.2, 'max_grad_norm': 0.5},
    2: {'n_steps': 10, 'ent_coef': 0.015, 'learning_rate': 0.0005, 'vf_coef': 0.3, 'max_grad_norm': 0.6},
    3: {'n_steps': 5, 'ent_coef': 0.02, 'learning_rate': 0.0007, 'vf_coef': 0.15, 'max_grad_norm': 0.7},
}

In [ ]:
date = datetime.now().strftime("%Y-%m-%d__%H-%M-%S")

name_model = f'A2C'

MODEL_DIR = os.path.join(ENV_DIR,name_model)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

for var in a2c_variants.keys():
    name_model_var= f'var_{var}'

    MODEL_DIR_VAR = os.path.join(MODEL_DIR,name_model_var)
    if not os.path.exists(MODEL_DIR_VAR):
        os.makedirs(MODEL_DIR_VAR)

    a2c_hyperparams = a2c_variants[var]
    model_a2c = A2C("MultiInputPolicy", env, verbose=1,**a2c_hyperparams)

    # Создайте директорию для записи данных TensorBoard
    log_dir = os.path.join(
        MODEL_DIR_VAR,
        date
        )  # Замените это на путь к вашей директории
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    # Создайте SummaryWriter
    writer = create_file_writer(log_dir)

    model_a2c.learn(
        total_timesteps=1e0, # Общее количество шагов времени, в течение которых будет произведено обучение.
        callback = CustomCallback(writer=writer),#  Обратный вызов (callback) для записи промежуточных результатов обучения.
        log_interval=1e2, #Интервал для записи логов.
        progress_bar=True
    )
    model_a2c.save(path=os.path.join(MODEL_DIR_VAR,f'PPO_{var}'))

### PPO model

In [ ]:
ppo_variants = {
    0: {
    'learning_rate' : 0.001,
    'n_steps' :2048,
    'ent_coef' : 0.01,
    'vf_coef' : 0.5
},
    1: {
    'learning_rate': 0.0005,
    'n_steps' : 1024,
    'ent_coef':  0.05,
    'vf_coef' : 0.5
},
    2: {
    'learning_rate'  : 0.002,
    'n_steps'  :  4096,
    'ent_coef'  : 0.005,
    'vf_coef' : 0.5
}
,
    3: {
        'learning_rate' : 0.001,
    'n_steps' : 2048,
    'ent_coef' : 0.1,
    'vf_coef' : 0.5
    }
}

In [ ]:
date = datetime.now().strftime("%Y-%m-%d__%H-%M-%S")

name_model = f'PPO'

MODEL_DIR = os.path.join(ENV_DIR,name_model)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)
    
for var in ppo_variants.keys():
    name_model_var = f'var_{var}'

    MODEL_DIR_VAR = os.path.join(MODEL_DIR,name_model_var)
    if not os.path.exists(MODEL_DIR_VAR):
        os.makedirs(MODEL_DIR_VAR)

    ppo_hyperparams = ppo_variants[var]
    model_ppo = PPO("MultiInputPolicy", env, verbose=1,**ppo_hyperparams)

    # Создайте директорию для записи данных TensorBoard
    log_dir = os.path.join(
        MODEL_DIR_VAR,
        date
        )  # Замените это на путь к вашей директории
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    # Создайте SummaryWriter
    writer = create_file_writer(log_dir)

    model_ppo.learn(
        total_timesteps=1,#e6, # Общее количество шагов времени, в течение которых будет произведено обучение.
        callback = CustomCallback(writer=writer),#  Обратный вызов (callback) для записи промежуточных результатов обучения.
        log_interval=1e2, #Интервал для записи логов.
        progress_bar=True
    )

### TD3 model

In [ ]:
td3_variants = {
    0: {
    "learning_rate": 5e-4,
    "buffer_size": 500_000,
    "learning_starts": 200,
    "batch_size": 64,
    "tau": 0.01,
    "gamma": 0.98,
    "train_freq": (2, "episode"),
    "gradient_steps": 50,
    "policy_delay": 3,
    "target_policy_noise": 0.15,
    "target_noise_clip": 0.4,
    "stats_window_size": 50,
}
,
    1: {
    "learning_rate": 1e-3,
    "buffer_size": 1_000_000,
    "learning_starts": 100,
    "batch_size": 100,
    "tau": 0.005,
    "gamma": 0.99,
    "train_freq": (4, "step"),
    "gradient_steps": 100,
    "policy_delay": 1,
    "target_policy_noise": 0.1,
    "target_noise_clip": 0.3,
    "stats_window_size": 200,
}
,
    2: {
    "learning_rate": 2e-4,
    "buffer_size": 750_000,
    "learning_starts": 150,
    "batch_size": 75,
    "tau": 0.0075,
    "gamma": 0.97,
    "train_freq": (3, "episode"),
    "gradient_steps": 75,
    "policy_delay": 2,
    "target_policy_noise": 0.125,
    "target_noise_clip": 0.35,
    "stats_window_size": 150,
}
,
    3: {
    "learning_rate": 3e-3,
    "buffer_size": 1_250_000,
    "learning_starts": 50,
    "batch_size": 50,
    "tau": 0.0025,
    "gamma": 0.95,
    "train_freq": (1, "step"),
    "gradient_steps": 200,
    "policy_delay": 4,
    "target_policy_noise": 0.2,
    "target_noise_clip": 0.45,
    "stats_window_size": 250,
}
}

In [ ]:
date = datetime.now().strftime("%Y-%m-%d__%H-%M-%S")

name_model = f'TD3'

MODEL_DIR = os.path.join(ENV_DIR,name_model)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)
    
for var in td3_variants.keys():
    name_model_var = f'var_{var}'

    MODEL_DIR_VAR = os.path.join(MODEL_DIR,name_model_var)
    if not os.path.exists(MODEL_DIR_VAR):
        os.makedirs(MODEL_DIR_VAR)
        
    td3_hyperparams = td3_variants[var]
    model_td3 = TD3("MultiInputPolicy", env, verbose=1,**td3_hyperparams)

    # Создайте директорию для записи данных TensorBoard
    log_dir = os.path.join(
        MODEL_DIR_VAR,
        date
        )  # Замените это на путь к вашей директории
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    # Создайте SummaryWriter
    writer = create_file_writer(log_dir)

    model_td3.learn(
        total_timesteps=1e0, # Общее количество шагов времени, в течение которых будет произведено обучение.
        callback = CustomCallback(writer=writer),#  Обратный вызов (callback) для записи промежуточных результатов обучения.
        log_interval=1e2, #Интервал для записи логов.
        progress_bar=True
    )
    model_td3.save(path=os.path.join(MODEL_DIR_VAR,f'TD3_{var}'))

## Visualize

#### Not trained

In [ ]:
a = env.optimal_space

In [ ]:
env._get_observation( env._model.get_output())['crop_model']['PAVAIL']

In [ ]:
env.reset()
def get_values(env):
    T_soil_arr = env.T_soil
    humid_soil_arr = env.humid_soil
    pH_arr = env.pH
    EC_arr = env.EC
    dV_arr = env.dV
    NAVAIL_arr = env._get_observation( env._model.get_output())['crop_model']['NAVAIL']
    PAVAIL_arr = env._get_observation( env._model.get_output())['crop_model']['PAVAIL']
    KAVAIL_arr = env._get_observation( env._model.get_output())['crop_model']['KAVAIL']
    return {
        'T_soil': T_soil_arr,
        'humid_soil': humid_soil_arr,
        'pH': pH_arr,
        'EC': EC_arr,
        'NAVAIL': np.array(NAVAIL_arr),
        'PAVAIL':  np.array(PAVAIL_arr),
        'KAVAIL': np.array(KAVAIL_arr),
        'dV':  np.array([ dV_arr])
    }
o_init = get_values(env)

In [ ]:
timesteps = 100
for _ in range(timesteps):
    a = env.action_space.sample()
    o, r, terminated, truncated, info = env.step(a)
    values = {
        'T_soil': env.T_soil,
        'humid_soil': env.humid_soil,
        'pH': env.pH,
        'EC': env.EC,
        'NAVAIL': np.array(o['crop_model']['NAVAIL']),
        'PAVAIL': np.array(o['crop_model']['PAVAIL']),
        'KAVAIL': np.array(o['crop_model']['KAVAIL']),
        'dV': np.array([env.dV])
    }
    for key in values.keys():
        if isinstance(o_init[key], np.ndarray):
            o_init[key] = np.append(o_init[key], values[key])
        # else:
        #     o_init[key] = o_init[key]

In [ ]:
for key in o_init.keys():
    print(o_init[key].shape)

In [ ]:
o_init_sh =o_init
for key in o_init.keys():
    o_init_sh[key] = o_init[key].reshape(1,-1)
    

In [ ]:
# Создаем временной ряд для оси X (в часах)
time_hours = np.linspace(0,timesteps,101)

# Создаем графики
fig, axs = plt.subplots(len(a), 1, figsize=(8, 6 * len(a)), sharex=True)

for i, (key, values) in enumerate(o_init.items()):
    if key  != 'dV':
        ax = axs[i]
        ax.set_title(key)

        # Горизонтальные линии из словаря a
        for value in a[key]:
            ax.axhline(y=value, color='r', linestyle='--', alpha=0.5)

        # Графики значений из словаря b
        ax.plot(time_hours, o_init[key])

        # Настройка меток осей
        ax.set_xlabel('Время')
        ax.set_ylabel(key)
        ax.grid()

# Устанавливаем общий заголовок для всех подграфиков
fig.suptitle('Горизонтальные линии и ряды значений')

plt.tight_layout()
plt.show()

#### Model trained

In [ ]:
model = SAC.load(path=os.path.join(os.getcwd(),'tensorboard_logs','GreenHousePCSE','SAC','var_0','SAC_0'),env=env)

In [ ]:
dir(model)

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy 

a = model.predict(env.reset()[0])[0]
env.step(a)[0]

In [ ]:
timesteps = 100
o,_ = env.reset()
# o = env.reshape_obs_dict(o)
obs = o

for i in range(timesteps):
    print(f'{i} {obs}')
    a,_ = model.predict(obs)
    obs, r, terminated, truncated, info = env.step(a)
    # for key in env._additional_variables:
    #     o['crop_model'][key] = o[key]
    #     del o[key]
    obs= env.reshape_obs_dict(obs)
    print(f'{i} {obs}')
    # values = {
    #     'T_soil': env.T_soil,
    #     'humid_soil': env.humid_soil,
    #     'pH': env.pH,
    #     'EC': env.EC,
    #     'NAVAIL': np.array(o['crop_model']['NAVAIL']),
    #     'PAVAIL': np.array(o['crop_model']['PAVAIL']),
    #     'KAVAIL': np.array(o['crop_model']['KAVAIL']),
    #     'dV': np.array([env.dV])
    # obs = env.reshape_obs_dict(obs)
    # }
    # for key in values.keys():
    #     if isinstance(o_init[key], np.ndarray):
    #         o_init[key] = np.append(o_init[key], values[key])
        # else:
        #     o_init[key] = o_init[key]